<a href="https://colab.research.google.com/github/itsap159/connect/blob/master/dsc210_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing the required modules
import pandas as pd
import numpy as np
import nltk
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('omw-1.4')
from nltk.corpus import wordnet
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#importing the dataset
news_data = pd.read_csv('/content/sample_data/news.csv')
#news_data= news_data[:500000]
news_data.head(5)

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [ ]:
headlines = news_data[['headline_text']]
headlines = headlines.astype('str')
print(headlines['headline_text'])
headlines.head(5)

0          aba decides against community broadcasting lic...
1             act fire witnesses must be aware of defamation
2             a g calls for infrastructure protection summit
3                   air nz staff in aust strike for pay rise
4              air nz strike to affect australian travellers
                                 ...                        
1244179    two aged care residents die as state records 2...
1244180    victoria records 5;919 new cases and seven deaths
1244181      wa delays adopting new close contact definition
1244182    western ringtail possums found badly dehydrate...
1244183    what makes you a close covid contact here are ...
Name: headline_text, Length: 1244184, dtype: object


,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


In [ ]:
def tokenize_words(txt):
  tokenized= word_tokenize(txt)
  return tokenized

In [ ]:
def remove_stopwords(tokenized_txt):
  cleaned_text_array = [word for word in tokenized_txt if word not in stop_words]
  return cleaned_text_array

In [ ]:
def word_lemmatizer(tokenized_text_without_stopwords):
  wordnet_lemmatizer = WordNetLemmatizer()
  lemmatized = [wordnet_lemmatizer.lemmatize(word) for word in tokenized_text_without_stopwords]
  return ' '.join(lemmatized)

In [ ]:
stop_words = stopwords.words('english')
headlines['headline_tokenized_text']=headlines['headline_text'].apply(tokenize_words)
headlines['processed_headlines'] = headlines['headline_tokenized_text'].apply(lambda x: remove_stopwords(x))

headlines['final_processed'] = headlines['processed_headlines'].apply(lambda x: word_lemmatizer(x))


In [ ]:
headlines.head(5)

,headline_text,headline_tokenized_text,processed_headlines,final_processed
0,aba decides against community broadcasting lic...,"[aba, decides, against, community, broadcastin...","[aba, decides, community, broadcasting, licence]",aba decides community broadcasting licence
1,act fire witnesses must be aware of defamation,"[act, fire, witnesses, must, be, aware, of, de...","[act, fire, witnesses, must, aware, defamation]",act fire witness must aware defamation
2,a g calls for infrastructure protection summit,"[a, g, calls, for, infrastructure, protection,...","[g, calls, infrastructure, protection, summit]",g call infrastructure protection summit
3,air nz staff in aust strike for pay rise,"[air, nz, staff, in, aust, strike, for, pay, r...","[air, nz, staff, aust, strike, pay, rise]",air nz staff aust strike pay rise
4,air nz strike to affect australian travellers,"[air, nz, strike, to, affect, australian, trav...","[air, nz, strike, affect, australian, travellers]",air nz strike affect australian traveller


In [ ]:
training_headlines = headlines['final_processed']

In [ ]:
vectorizer = CountVectorizer(analyzer='word', max_features=5000)
x_counts = vectorizer.fit_transform(training_headlines)

In [ ]:
transformer = TfidfTransformer(smooth_idf=False)
x_tfidf = transformer.fit_transform(x_counts)

In [ ]:
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)

In [16]:
#obtain a NMF model.
num_topics = 8
model = NMF(n_components=num_topics, init='nndsvd');
#fit the model
model.fit(xtfidf_norm)

NMF(init='nndsvd', n_components=8)

In [17]:
def get_nmf_topics(model, n_top_words):
    
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(num_topics):
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict)

In [18]:
get_nmf_topics(model, 20)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08
0,interview,man,new,abc,police,fire,rural,win
1,extended,charged,council,weather,probe,house,national,australia
2,michael,murder,say,sport,investigate,home,news,cup
3,david,court,plan,entertainment,death,crew,nsw,world
4,james,dy,call,business,search,damage,qld,open
5,ben,woman,back,news,missing,warning,sa,australian
6,john,crash,water,market,officer,threat,reporter,day
7,andrew,face,govt,analysis,car,suspicious,closer,gold
8,matt,death,get,speaks,crash,destroys,woman,final
9,nrl,accused,report,talk,hunt,school,market,award
